In [1]:
#convert python txt <p1> <p2> file to json file format
import json
f = open("mapping_db_wk_clean.txt",'r',encoding='utf-8').read()
json_dict = {"values" : []}
for line in f.split("\n"):
    values = line.split(" ")
    if len(values) == 2 : db,wk = values
    json_dict["values"].append({"db" : db, "wk": wk})
with open("mapping_db_wk.json", "w") as outfile:
    json.dump(json_dict, outfile)

In [2]:
import json
#compare sameAs prop with clean db prop
with open('mapping_db_wk.json', 'r') as openfile:
    mapping_dict = json.load(openfile)

#load mapping into a dict to get constant acces time
db_sameAs_dict = {}
for i in mapping_dict["values"]:
    db_sameAs_dict[i["db"]] = i["wk"]

with open("../data/property_support.json") as json_file:
        property_dictionnary = json.load(json_file)
            


In [3]:
#filter prop count dictionary with sameAs dictionary
filtered_prop = {}
for i in db_sameAs_dict.keys():
    val = property_dictionnary.get(i)
    if val: filtered_prop[i] = val

#sort the dict
prop = dict(sorted(filtered_prop.items(), key=lambda item: item[1], reverse=True))
#keep only prop with support > 1000
prop = dict(filter(lambda x,: True if x[1] > 1000 else False, prop.items()))



In [4]:
db_sameAs_dict['<http://dbpedia.org/ontology/deathDate>']

'<http://www.wikidata.org/prop/P570>'

In [5]:
for i in prop.keys():
        print(f"{db_sameAs_dict[i]} sameAs {i} : {prop[i]}")


<http://www.wikidata.org/prop/P11143> sameAs <http://dbpedia.org/ontology/wikiPageID> : 1406259
<http://www.wikidata.org/prop/P2451> sameAs <http://dbpedia.org/ontology/birthDate> : 471625
<http://www.wikidata.org/prop/P570> sameAs <http://dbpedia.org/ontology/deathDate> : 170365
<http://www.wikidata.org/prop/P2907> sameAs <http://dbpedia.org/ontology/utcOffset> : 163232
<http://www.wikidata.org/prop/P6499> sameAs <http://dbpedia.org/ontology/populationTotal> : 154981
<http://www.wikidata.org/prop/P8497> sameAs <http://dbpedia.org/ontology/Person/height> : 111006
<http://www.wikidata.org/prop/P8483> sameAs <http://dbpedia.org/ontology/postalCode> : 106248
<http://www.wikidata.org/prop/P5286> sameAs <http://dbpedia.org/ontology/runtime> : 97818
<http://www.wikidata.org/prop/P1477> sameAs <http://dbpedia.org/ontology/originalName> : 70857
<http://www.wikidata.org/prop/P10708> sameAs <http://dbpedia.org/ontology/areaCode> : 66970
<http://www.wikidata.org/prop/P10673> sameAs <http://dbpedi

### Test values 

In [6]:
#import 
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import re

In [7]:
#const
DBPEDIA = "http://dbpedia.org/sparql"
WIKIDATA = "https://query.wikidata.org/sparql"

In [8]:
#isbn - wdt:P957

In [9]:
def sparql_query(query : str, endpoint : str ) -> dict:
    sparql = SPARQLWrapper(
        endpoint
    )
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)

    try:
        ret = sparql.queryAndConvert()
        return ret
    except Exception as e:
        print(e)
        raise Exception("Error in query")

In [10]:
def query_value_sample(prop : str) -> str:
    query = """
                select ?e ?p ?v where {
                    values ?p {"""+prop+"""}.
                    ?e ?p ?v
                } limit 100
            """
    query = re.sub(r"\n|'","",query)
    return query


In [11]:
#find prop to query
db_prop = "<http://dbpedia.org/ontology/releaseDate>"
wk_prop = "<http://www.wikidata.org/prop/statement/P10673>"
result_db = sparql_query(query_value_sample(db_prop),DBPEDIA)
result_wk = sparql_query(query_value_sample(wk_prop),WIKIDATA)

In [12]:
date_i = []
for i,j in zip(result_db["results"]["bindings"],result_wk["results"]["bindings"]):
    #print(i)
    #print(i['v']['value'],j['v']['value'])
    date_i.append(i['v']['value'])

In [13]:
date_i

['2006-10-28',
 '2005-11-15',
 '1936-11-07',
 '1955-09-08',
 '1936-12-07',
 '1937-04-12',
 '1974-12-02',
 '1972-06-30',
 '1955-07-06',
 '2006-09-09',
 '2007-11-06',
 '2007-11-13',
 '2009-09-29',
 '2014-03-25',
 '2001-12-04',
 '2004-09-06',
 '2004-11-10',
 '2004-11-23',
 '2004-12-09',
 '2005-09-27',
 '2011-09-27',
 '2000-09-12',
 '1999-09-14',
 '2003-11-11',
 '2004-02-05',
 '2006-04-04',
 '2005-11-16',
 '2005-11-24',
 '2008-09-23',
 '2010-10-19',
 '2005-09-13',
 '2005-09-14',
 '2006-03-07',
 '2009-09-08',
 '2011-11-01',
 '2003-06-03',
 '1969-02-10',
 '2017-04-28',
 '2022-10-24',
 '2004-05-10',
 '2018-03-14',
 '2002-04-11',
 '2013-03-09',
 '2012-06-11',
 '1992-11-21',
 '2001-09-23',
 '2017-05-23']

In [14]:
from datetime import datetime

dates = [
    '2006-10-28', '2005-11-15', '1936-11-07', '1955-09-08', '1936-12-07',
    '1937-04-12', '1974-12-02', '1972-06-30', '1955-07-06', '2006-09-09',
    '2007-11-06', '2007-11-13', '2009-09-29', '2014-03-25', '2001-12-04',
    '2004-09-06', '2004-11-10', '2004-11-23', '2004-12-09', '2005-09-27',
    '2011-09-27', '2000-09-12', '1999-09-14', '2003-11-11', '2004-02-05',
    '2006-04-04', '2005-11-16', '2005-11-24', '2008-09-23', '2010-10-19',
    '2005-09-13', '2005-09-14', '2006-03-07', '2009-09-08', '2011-11-01',
    '2003-06-03', '1969-02-10', '2017-04-28', '2022-10-24', '2004-05-10',
    '2018-03-14', '2002-04-11', '2013-03-09'
]

american_dates = []
european_dates = []

for date_str in dates:
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    american_date = date_obj.strftime('%m/%d/%Y')
    american_dates.append(american_date)
    european_date = date_obj.strftime('%d/%m/%Y')
    european_dates.append(european_date)

# Print the converted dates
for a,e in zip(american_dates,european_dates):
    print(a+" - "+e)


10/28/2006 - 28/10/2006
11/15/2005 - 15/11/2005
11/07/1936 - 07/11/1936
09/08/1955 - 08/09/1955
12/07/1936 - 07/12/1936
04/12/1937 - 12/04/1937
12/02/1974 - 02/12/1974
06/30/1972 - 30/06/1972
07/06/1955 - 06/07/1955
09/09/2006 - 09/09/2006
11/06/2007 - 06/11/2007
11/13/2007 - 13/11/2007
09/29/2009 - 29/09/2009
03/25/2014 - 25/03/2014
12/04/2001 - 04/12/2001
09/06/2004 - 06/09/2004
11/10/2004 - 10/11/2004
11/23/2004 - 23/11/2004
12/09/2004 - 09/12/2004
09/27/2005 - 27/09/2005
09/27/2011 - 27/09/2011
09/12/2000 - 12/09/2000
09/14/1999 - 14/09/1999
11/11/2003 - 11/11/2003
02/05/2004 - 05/02/2004
04/04/2006 - 04/04/2006
11/16/2005 - 16/11/2005
11/24/2005 - 24/11/2005
09/23/2008 - 23/09/2008
10/19/2010 - 19/10/2010
09/13/2005 - 13/09/2005
09/14/2005 - 14/09/2005
03/07/2006 - 07/03/2006
09/08/2009 - 08/09/2009
11/01/2011 - 01/11/2011
06/03/2003 - 03/06/2003
02/10/1969 - 10/02/1969
04/28/2017 - 28/04/2017
10/24/2022 - 24/10/2022
05/10/2004 - 10/05/2004
03/14/2018 - 14/03/2018
04/11/2002 - 11/

In [25]:
f = open("../data/prop_releaseDate_support_db.ttl","r",encoding="utf-8").read()
for i in f.split("\n"):
    e,p,v = i.split("\t")
    break

In [19]:
foo = [set() for i in range(2)]
with open("toot.txt","r") as f:
    for i in f.readlines():
        ii = i.split("_")
        foo[0].add(ii[0])
        foo[1].add(ii[1])
print(foo)

[{'510', '123', '789'}, {'lala\n', 'lk\n'}]


In [30]:
a ="<http://dbpedia.org/prop/P110>"

"/".join((x:= a.split("/"))[0:-1])+"/statement/"+x[-1]

'<http://dbpedia.org/prop/statement/P110>'

In [ ]:
import typing
from functools import reduce 
import ijson
import json
import os

def property_list_cutting(l: list ,slice_size: int ) -> list[list[str]]:
    """From a list, return slice 2D with n sublist of size "slice_size"

    Args:
        l (_type_): huge 1D list
        slice_size (_type_): 2D list composed of n "slice_size" list

    Returns:
        list : slice_size
    """
    index = 0
    sliced_list = []
    if len(l) < slice_size: return l
    while index<len(l):
        sliced_list.append(l[index:index+slice_size])
        index+=slice_size
    return sliced_list

def query_generate_VALUES(slice: list) -> str:
    """From a list return a string composed of all element of the list joined

    Args:
        slice (list): 1D list

    Returns:
        str : return a string of the type " slice[0] slice[1] slice[2] ... "
    """
    func = lambda acc,x : acc+" "+str(x)
    acc = ""
    joined_string = reduce(func, slice, acc)
    return joined_string

#TODO
def read_sameAs_file(file: str, var_names : list[str] = ["DB","WK"]) -> tuple[set[str],set[str]]:
    """Generate a list of sameAs from Dbpedia and Wikidata
    Args:
        file (str): sameAs json file name
    """
    f_read = open(file, 'r', encoding="UTF-8")

    first,second = var_names#refactor with var name header from json file

    db_entity_list = set()
    wk_entity_list = set()

    for record in ijson.items(f_read, "results.bindings.item"):
        db_entity_list.add(f'<{record[first]["value"]}>')
        wk_entity_list.add(f'<{record[second]["value"]}>')

    f_read.close()
    return db_entity_list,wk_entity_list


def read_ttl_file(file: str) -> tuple[list[str],list[str],list[str]]:
    """Generate a list entity property value from a ttl file
    Args:
        file (str): sameAs json file name
    """
    f_read = open(file, 'r', encoding="UTF-8").read()

    entity = []
    prop = []
    value = []

    for line in f_read.split("\n"):
        _line = line.split("\t")
        if len(_line) == 3:
            e,p,v = _line
            entity.append(e)
            prop.append(p)
            value.append(v)
        
    return entity,prop,value


def read_result_file(file : str, output_file : str, prop_var_name : str) -> None:
    """Read a result file to fill a dictionary 

    Args:
        file (str): result file in json format
    """
    
    f_read = open(file, 'r', encoding="UTF-8")
    prop = prop_var_name #name of the sparql var
    properties_count_file = output_file #dict file name
    #if file exist open json file and load 
    if os.path.exists(properties_count_file):
        with open(properties_count_file) as json_file:
            property_dictionnary = json.load(json_file)
            
    #else start with empty dict
    else:
        property_dictionnary = {}

    for record in ijson.items(f_read, "results.bindings.item"):
        item = (f'<{record[prop]["value"]}>')
        prop_count = property_dictionnary.get(item)
        if prop_count: property_dictionnary[item] += 1
        else: property_dictionnary[item] = 1

    f_read.close()
    #dump new dict
    with open(properties_count_file, "w") as fp:
        json.dump(property_dictionnary,fp) 

def sparql_call(sparql_query: str, result_file : str, database_name : str = "dbpedia") -> int :
    """Call a jar file to execute a sparql query on a database for a specified query 

    Args:
        sparql_query (str): sparql query we want to execute on a database
    """
    jar_path = "~/../soulard/QueryHDT/SparqlToJSON.jar"
    database_path = {
        "dbpedia" : "~/../soulard/Graphs_HDT/DBpedia/DBpedia_en.hdt",
        "wikidata" : "~/../soulard/Graphs_HDT/Wikidata/Wikidata_final.hdt" 
    }
    dataset_path = database_path[database_name]
    #hdt_command = "nohup java -Xmx120G -Xms120G -jar "+jar_path+" "+dataset_path+" \""+sparql_query+"\" > "+result_file+" &"
    hdt_command = "java -jar "+jar_path+" "+dataset_path+" \""+sparql_query+"\" > "+result_file
    return os.system(hdt_command)

def clean_nohup_file(result_file : str) -> None:
    """Commands to remove unwanted string at the start of nohup resulting files

    Args:
        result_file (str): _description_

    Returns:
        _type_: _description_
    """
    code = 0
    new_res_file = result_file.split(".")[0]+"_clean.json"
    code += os.system("cp "+result_file+" "+new_res_file)
    code += os.system("awk 'NR > 3 { print }' < "+new_res_file+" > "+result_file) #remove first 3 lines of the file
    return code

def concat_result_files(result_file : str, output_file : str, var_names : list) -> None:
    """Concat the json file to the output file

    Args:
        result_file (str): json file 
        output_file (str): ttl format file, append at the end of file
    """
    
    e,p,v = var_names
    f_read = open(result_file, 'r', encoding="UTF-8")
    f_write = open(output_file, 'a', encoding="UTF-8")
    for record in ijson.items(f_read, "results.bindings.item"):
        entity = (f'<{record[e]["value"]}>')
        prop = (f'<{record[p]["value"]}>')
        value = (f'{record[v]["value"]}')
        f_write.write(entity+"\t"+prop+"\t"+value+"\n")
        
    f_read.close()
    f_write.close()



